In [1]:
import netCDF4
import pandas
import datetime
import numpy

In [2]:
minCutoffFitLat = 45.
delTimeCutOffNrstPass = 45 # min
mlonDiffOtrEndCutoff = 50.
delLatCutoff = 2.
delCtimeCutoff = 60. #min

In [3]:
fileList = [ "../poes_n15_20130302_proc.nc",\
                "../poes_n18_20130302_proc.nc",\
                "../poes_m01_20130302_proc.nc",\
                "../poes_n19_20130302_proc.nc",\
                "../poes_m01_20130302_proc.nc",\
                "../poes_n16_20130302_proc.nc",\
                "../poes_n17_20130302_proc.nc" ]
poesAllEleDataDF = pandas.DataFrame( columns =  ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_ele_flux", "sat"] )
poesAllProDataDF = pandas.DataFrame( columns =  ["timestamp", "date", "aacgm_lat_foot",\
                         "aacgm_lon_foot", "MLT", "log_pro_flux", "sat"] )
for f in fileList:
    print "reading file-->", f
    # read variable from the netCDF files
    poesRawData = netCDF4.Dataset(f)
    poesDF = pandas.DataFrame( poesRawData.variables['time'][:], columns=[ "timestamp" ] )
    poesDF['date'] = pandas.to_datetime(poesDF['timestamp'], unit='ms')
    poesDF["alt"] = poesRawData.variables['alt'][:]
    poesDF["aacgm_lat_foot"] = poesRawData.variables['aacgm_lat_foot'][:]

    poesDF["aacgm_lon_foot"] = poesRawData.variables['aacgm_lon_foot'][:]
    poesDF["MLT"] = poesRawData.variables['MLT'][:]
    # round of to 2 decimals
    poesDF['alt'] = [ round( x, 2 ) for x in poesDF['alt']]
    poesDF['aacgm_lat_foot'] = [ round( x, 2 ) for x in poesDF['aacgm_lat_foot']]
    poesDF['aacgm_lon_foot'] = [ round( x, 2 ) for x in poesDF['aacgm_lon_foot']]
    poesDF['MLT'] = [ round( x, 2 ) for x in poesDF['MLT']]
    # Add up the fluxes
    poesDF["ted_ele_total_flux"] = poesRawData.variables['ted_ele_tel0_flux_4'][:] +\
            poesRawData.variables['ted_ele_tel0_flux_8'][:] + \
            poesRawData.variables['ted_ele_tel0_flux_11'][:] + \
            poesRawData.variables['ted_ele_tel0_flux_14'][:] + \
            poesRawData.variables['ted_ele_tel30_flux_4'][:] +\
            poesRawData.variables['ted_ele_tel30_flux_8'][:] + \
            poesRawData.variables['ted_ele_tel30_flux_11'][:] + \
            poesRawData.variables['ted_ele_tel30_flux_14'][:]
    poesDF["ted_pro_total_flux"] = poesRawData.variables['ted_pro_tel0_flux_4'][:] +\
            poesRawData.variables['ted_pro_tel0_flux_8'][:] + \
            poesRawData.variables['ted_pro_tel0_flux_11'][:] + \
            poesRawData.variables['ted_pro_tel0_flux_14'][:] + \
            poesRawData.variables['ted_pro_tel30_flux_4'][:] +\
            poesRawData.variables['ted_pro_tel30_flux_8'][:] + \
            poesRawData.variables['ted_pro_tel30_flux_11'][:] + \
            poesRawData.variables['ted_pro_tel30_flux_14'][:]
    poesDF['log_ele_flux'] = poesDF['ted_ele_total_flux']#[0. if x <= 0. else round( numpy.log10(x), 2 )\
#                  for x in poesDF['ted_ele_total_flux']]
    poesDF['log_pro_flux'] = poesDF['ted_pro_total_flux']#[0. if x <= 0. else round( numpy.log10(x), 2 )\
#                  for x in poesDF['ted_pro_total_flux']]
    # the current satellite number
    poesDF["sat"] = f[-19:-17]
#     seperate out electron and proton flux and discard all zeros
    currPoesEleFluxDF = poesDF[poesDF["log_ele_flux"] > 0.][ ["timestamp",\
                     "date", "aacgm_lat_foot", "aacgm_lon_foot", "MLT",\
                     "log_ele_flux", "sat"] ].reset_index(drop=True)
    currPoesProFluxDF = poesDF[poesDF["log_pro_flux"] > 0.][ ["timestamp",\
                     "date", "aacgm_lat_foot", "aacgm_lon_foot", "MLT",\
                     "log_pro_flux", "sat"] ].reset_index(drop=True)
    
    
    poesAllEleDataDF = poesAllEleDataDF.append( currPoesEleFluxDF )
    poesAllProDataDF = poesAllProDataDF.append( currPoesProFluxDF )
    # now delete all the rows for prev DFs
    # we don't want to duplicate data
    poesDF = poesDF.drop( poesDF.index )
    currPoesEleFluxDF = currPoesEleFluxDF.drop( currPoesEleFluxDF.index )
    currPoesProFluxDF = currPoesProFluxDF.drop( currPoesProFluxDF.index )
# create a date and time columns
poesAllEleDataDF["dateStr"] = poesAllEleDataDF["date"].map(lambda x: x.strftime('%Y%m%d'))
poesAllEleDataDF["time"] = poesAllEleDataDF["date"].map(lambda x: x.strftime('%H%M'))
poesAllProDataDF["dateStr"] = poesAllProDataDF["date"].map(lambda x: x.strftime('%Y%m%d'))
poesAllProDataDF["time"] = poesAllProDataDF["date"].map(lambda x: x.strftime('%H%M'))

reading file--> ../poes_n15_20130302_proc.nc
reading file--> ../poes_n18_20130302_proc.nc
reading file--> ../poes_m01_20130302_proc.nc
reading file--> ../poes_n19_20130302_proc.nc
reading file--> ../poes_m01_20130302_proc.nc
reading file--> ../poes_n16_20130302_proc.nc
reading file--> ../poes_n17_20130302_proc.nc


In [4]:
timeRange = [ poesAllEleDataDF["date"].min(), poesAllEleDataDF["date"].max() ]
ctime = timeRange[0]
timeInterval=datetime.timedelta(minutes=30)
while ctime <= timeRange[1]:
    ctime += timeInterval
    if abs( ctime - datetime.datetime(2013,3,2,3) ) < datetime.timedelta(minutes=1):
        break
print ctime

2013-03-02 03:00:00.201000


In [5]:
poesAllEleDataDF = poesAllEleDataDF[ [ 'sat', 'date',\
                        'aacgm_lat_foot', 'aacgm_lon_foot',\
                            'MLT', 'log_ele_flux' ] ]
poesAllEleDataDF["delCtime"] = abs(poesAllEleDataDF["date"] - ctime)
poesAllEleDataDF["delLatFit"] = abs( poesAllEleDataDF["aacgm_lat_foot"] ) -\
                                    abs( minCutoffFitLat )
# We are sorting by sats, dates and lats to pick the nearest time
# when the satellite is between two 45 MLATs
poesAllEleDataDFNth = poesAllEleDataDF[ poesAllEleDataDF["aacgm_lat_foot"]\
                        >= 0. ].sort_values( ['sat', 'date', 'aacgm_lat_foot'],\
                                ascending=True ).reset_index(drop=True).drop_duplicates()
poesAllEleDataDFSth = poesAllEleDataDF[ poesAllEleDataDF["aacgm_lat_foot"]\
                        < 0. ].sort_values( ['sat', 'date', 'aacgm_lat_foot'],\
                                ascending=True ).reset_index(drop=True).drop_duplicates()

In [8]:
delTTest = datetime.timedelta(minutes=37)
print poesAllEleDataDFNth[ \
        (poesAllEleDataDFNth["sat"] == "15") &\
        (poesAllEleDataDFNth["date"] > ctime+delTTest)\
        ][ [ "sat", "date", "aacgm_lat_foot" ] ].sort_values( ["date"] )

      sat                    date  aacgm_lat_foot
16987  15 2013-03-02 03:37:04.714           38.17
16988  15 2013-03-02 03:37:06.714           38.26
16989  15 2013-03-02 03:37:12.714           38.53
16990  15 2013-03-02 03:37:14.714           38.62
16991  15 2013-03-02 03:37:20.713           38.90
16992  15 2013-03-02 03:37:22.713           38.99
16993  15 2013-03-02 03:37:28.714           39.26
16994  15 2013-03-02 03:37:30.714           39.35
16995  15 2013-03-02 08:58:00.714           78.36
16996  15 2013-03-02 08:58:02.714           78.32
16997  15 2013-03-02 08:58:08.714           78.21
16998  15 2013-03-02 08:58:10.714           78.17
16999  15 2013-03-02 08:58:16.713           78.04
17000  15 2013-03-02 08:58:18.713           78.00
17001  15 2013-03-02 08:58:24.713           77.86
17002  15 2013-03-02 08:58:26.714           77.81
17003  15 2013-03-02 08:58:32.714           77.66
17004  15 2013-03-02 08:58:34.714           77.61
17005  15 2013-03-02 08:58:40.713           77.45
